In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
train_data = pd.read_json(r'../input/train.json')
test_data = pd.read_json(r'../input/test.json')
print(train_data.columns)
print(test_data.columns)

                       

In [ ]:
print("shape of training data: \n")
print(train_data.shape)
print("\n")
print("shape of test data: \n")
print(test_data.shape)
print("\n")

In [ ]:

train_data.groupby(['interest_level'])['listing_id'].count().plot(kind = 'bar')
plt.ylabel('number of observations')

In [ ]:
import matplotlib.pyplot as plt
cols = train_data.columns
other_cols = []
winsorize_limits = [0.01,0.99]
i = 0
for c in cols:
    if (type(list(train_data[c])[0])==np.int64)|(type(list(train_data[c])[0])==np.float64):
        i=i+1
        plt.figure(i)
        train_data.loc[(train_data[c]>train_data[c].quantile(winsorize_limits[0]))&(train_data[c]<train_data[c].quantile(winsorize_limits[1])),c].hist()
        print("Max " + c + " : " + str(train_data[c].max()))
        print("Min " + c + " : " + str(train_data[c].min()))
        plt.title('Histogram for winsorized ' + c)

In [ ]:
cols = train_data.columns
other_cols = []
winsorize_limits = [0.01,0.99]
i = 0
for c in cols:
    if (type(list(train_data[c])[0])==np.int64)|(type(list(train_data[c])[0])==np.float64):
        i=i+1
        plt.figure(i)
        train_data_temp = train_data[(train_data[c]>train_data[c].quantile(winsorize_limits[0]))&(train_data[c]<train_data[c].quantile(winsorize_limits[1]))]
        df_temp = pd.DataFrame(train_data_temp.groupby(['interest_level'])[c].mean())
        df_temp.plot(kind = 'bar')
        plt.title('Average winsorized ' + c + ' by interest_level ' )

In [ ]:
temp_cols = ['latitude','longitude']
train_data_temp = train_data.copy(deep = True)
for c in temp_cols:
    train_data_temp = train_data_temp[(train_data_temp[c]>train_data_temp[c].quantile(winsorize_limits[0]))&(train_data_temp[c]<train_data_temp[c].quantile(winsorize_limits[1]))]
grp = train_data_temp.groupby(['interest_level'])
fig,ax = plt.subplots()
for name,group in grp:
    ax.plot(group.latitude,group.longitude,marker = 'o',label = name,linestyle = '',ms = 3)
ax.legend()
plt.xlabel('Latitude')
plt.ylabel('Longitude')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

In [ ]:
temp_cols = ['latitude','longitude']
train_data_temp = train_data.copy(deep = True)
for c in temp_cols:
    train_data_temp = train_data_temp[(train_data_temp[c]>train_data_temp[c].quantile(winsorize_limits[0]))&(train_data_temp[c]<train_data_temp[c].quantile(winsorize_limits[1]))]
grp = train_data_temp.groupby(['interest_level'])
i = 0
for name,group in grp:
    i = i + 1
    plt.figure(i)
    if name == 'high':
        color_name = 'blue'
    if name == 'medium':
        color_name = 'green'
    if name == 'low':
        color_name = 'brown'
    plt.plot(group.latitude,group.longitude,marker = 'o',label = name,linestyle = '',ms = 3,color = color_name)
    plt.xlabel('Latitude')
    plt.ylabel('Longitude')
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    

In [ ]:
temp_cols = ['price','bedrooms']
winsorize_limits = [0.01,0.99]
train_data_temp = train_data.copy(deep = True)
for c in temp_cols:
    train_data_temp = train_data_temp[(train_data_temp[c]>train_data_temp[c].quantile(winsorize_limits[0]))&(train_data_temp[c]<train_data_temp[c].quantile(winsorize_limits[1]))]
grp = train_data_temp.groupby(['interest_level'])
fig,ax = plt.subplots()
for name,group in grp:
    if name=='high':
        ax.plot(group.price,group.bedrooms,marker = 'x',label = name,linestyle = '',ms = 12)
    else:
        ax.plot(group.price,group.bedrooms,marker = 'o',label = name,linestyle = '',ms = 3)
    
ax.legend()
plt.xlabel('price')
plt.ylabel('bedrooms')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)


train_data['ratio_bedroom_by_price'] = train_data['bedrooms']/train_data['price']
plt.figure(2)
train_data.groupby(['interest_level'])['ratio_bedroom_by_price'].mean().plot(kind = 'bar')

In [ ]:
from sklearn.neural_network import MLPClassifier
train_data['interest_level_int'] = ((train_data['interest_level']=='high')*3) +((train_data['interest_level']=='medium')*2)+((train_data['interest_level']=='low')*1) 
train_data['ratio_bedroom_by_price'] = train_data['bedrooms']/train_data['price']
winsorize_limits = [0.01,0.99]
cols = list(train_data.columns)
cols.remove('listing_id')
int_float_cols = []
cross_validation_data = train_data.sample(frac = 0.2)
train_data_temp = train_data.drop(cross_validation_data.index)

#winsorize all columns
for c in cols:
    if c == 'interest_level_int':
        continue
    if (type(list(train_data_temp[c])[0])!=str)&(type(list(train_data_temp[c])[0])!=list):
        int_float_cols.append(c)
        idx_lower_bound = train_data_temp[c]<train_data_temp[c].quantile(winsorize_limits[0])
        idx_higher_bound = train_data_temp[c]>train_data_temp[c].quantile(winsorize_limits[1])
        train_data_temp.loc[idx_lower_bound,c] = train_data_temp[c].quantile(winsorize_limits[0])
        train_data_temp.loc[idx_higher_bound,c] = train_data_temp[c].quantile(winsorize_limits[1])
        

Y = train_data_temp['interest_level_int'].as_matrix()
i = 0
#standardize all X variables
for c in int_float_cols:
    i = i + 1
    train_data_temp[c] = (train_data_temp[c] - np.nanmean(train_data_temp[c]))/(np.nanstd(train_data_temp[c]))
                          
X = train_data_temp[int_float_cols].as_matrix()

print("Training neural nets... ")
my_classifier = MLPClassifier(solver = 'adam',hidden_layer_sizes = (5,4),activation = 'logistic')
my_classifier.fit(X,Y)

In [ ]:
int_float_cols

In [ ]:
from sklearn.metrics import accuracy_score
y_variable = 'interest_level_int'
int_float_cols = []
#winsorize all CV columns
for c in cols:
    if c == 'interest_level_int':
        continue
    if (type(list(cross_validation_data[c])[0])!=str)&(type(list(cross_validation_data[c])[0])!=list):
        int_float_cols.append(c)
        idx_lower_bound = cross_validation_data[c]<cross_validation_data[c].quantile(winsorize_limits[0])
        idx_higher_bound = cross_validation_data[c]>cross_validation_data[c].quantile(winsorize_limits[1])
        cross_validation_data.loc[idx_lower_bound,c] = cross_validation_data[c].quantile(winsorize_limits[0])
        cross_validation_data.loc[idx_higher_bound,c] = cross_validation_data[c].quantile(winsorize_limits[1])
        

Y_cv = cross_validation_data[y_variable].as_matrix()
i = 0
#standardize all X CV variables
for c in int_float_cols:
    i = i + 1
    cross_validation_data[c] = (cross_validation_data[c] - np.nanmean(cross_validation_data[c]))/(np.nanstd(cross_validation_data[c]))
                          
X_cv = cross_validation_data[int_float_cols].as_matrix()


predicted = my_classifier.predict(X_cv)

print("Getting Accuracy for CV...")
accu = accuracy_score(Y_cv,predicted)
print("Accuracy = "+str(accu))



In [ ]:
from sklearn import linear_model
Y = train_data_temp['interest_level_int'].as_matrix()
X = train_data_temp[int_float_cols].as_matrix()

print("Logistic Regression... ")

logreg = linear_model.LogisticRegression(C=1e5,solver = 'sag')
logreg.fit(X, Y)
predicted = logreg.predict(X_cv)
accu = accuracy_score(Y_cv,predicted)
print("Accuracy = "+str(accu))

In [ ]:
len(X_cv[0])